In [ ]:
import os

# Set the working directory
working_directory = os.path.join(os.path.expanduser("~"), "Library", "CloudStorage", 
                              "OneDrive-UniversityofExeter", "Documents", "VISIONARY", 
                              "Durham Experiment", "processed_data_3")
os.chdir(working_directory)

# Verify the current working directory
print(f"Current working directory: {os.getcwd()}")

In [ ]:
from pathlib import Path

# Get the .mp4 files in the folder
mp4_files = list(Path(working_directory).glob("*.mp4"))
# Print the .mp4 files without showing the parent directories
print([file.name for file in mp4_files])

In [ ]:
# Filter the files for Camera_1 and Camera_2
camera_1_files = [file for file in mp4_files if file.name.startswith("Camera_1_")]
camera_2_files = [file for file in mp4_files if file.name.startswith("Camera_2_")]

# Sort the files by date extracted from the filename
camera_1_files_sorted = sorted(camera_1_files, key=lambda x: x.stem.split('_')[-1])
camera_2_files_sorted = sorted(camera_2_files, key=lambda x: x.stem.split('_')[-1])

print("Camera 1 files sorted by date:", [file.name for file in camera_1_files_sorted])
print("Camera 2 files sorted by date:", [file.name for file in camera_2_files_sorted])

del mp4_files, camera_1_files, camera_2_files

In [ ]:
import cv2
import json
import numpy as np

def draw_door_area(video_path, save_path):
    # Read the first frame of the video
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    # Create a copy of the original frame
    original_frame = frame.copy()
    drawing_frame = frame.copy()
    
    # Store the coordinates of the rectangle
    door_coords = []
    
    def mouse_callback(event, x, y, flags, param):
        nonlocal drawing_frame
        
        if event == cv2.EVENT_LBUTTONDOWN:
            # Reset if we already have 2 points
            if len(door_coords) == 2:
                door_coords.clear()
                drawing_frame = original_frame.copy()
            
            door_coords.append((x, y))
            # Draw point for visual feedback
            cv2.circle(drawing_frame, (x, y), 3, (0, 255, 0), -1)
            
            if len(door_coords) == 2:
                # Draw rectangle using the two points
                cv2.rectangle(drawing_frame, door_coords[0], door_coords[1], (0, 255, 0), 2)
                # Add instruction text
                cv2.putText(drawing_frame, "Press 'r' to redraw, Enter to confirm", 
                          (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow('Frame', drawing_frame)

    # Create window and set mouse callback
    cv2.imshow('Frame', drawing_frame)
    cv2.setMouseCallback('Frame', mouse_callback)
    
    # Instructions
    print("Instructions:")
    print("1. Click two points to draw a rectangle")
    print("2. Press 'r' to redraw if needed")
    print("3. Press Enter to confirm and save")
    print("4. Press 'q' to quit without saving")
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # Press 'r' to reset
        if key == ord('r'):
            door_coords.clear()
            drawing_frame = original_frame.copy()
            cv2.imshow('Frame', drawing_frame)
        
        # Press Enter to confirm
        elif key == 13:  # Enter key
            if len(door_coords) == 2:
                # Ensure coordinates are in top-left, bottom-right order
                x1, y1 = min(door_coords[0][0], door_coords[1][0]), min(door_coords[0][1], door_coords[1][1])
                x2, y2 = max(door_coords[0][0], door_coords[1][0]), max(door_coords[0][1], door_coords[1][1])
                
                coordinates = {
                    'top_left': {'x': x1, 'y': y1},
                    'bottom_right': {'x': x2, 'y': y2}
                }
                with open(save_path, 'w') as f:
                    json.dump(coordinates, f)
                print(f"\nDoor coordinates saved to {save_path}:")
                print(f"Top-left: ({x1}, {y1})")
                print(f"Bottom-right: ({x2}, {y2})")
                break
            else:
                print("Please complete the rectangle before confirming")
        
        # Press 'q' to quit
        elif key == ord('q'):
            print("\nQuitting without saving")
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Usage example
video_path = camera_1_files_sorted[0]
draw_door_area(video_path, save_path="door_coordinates_camera1.json")

#video_path = camera_2_files_sorted[0]
#draw_door_area(video_path, save_path="door_coordinates_camera2.json")

In [ ]:
Door coordinates saved to door_coordinates_camera1.json:
        Top-left: (1033, 9)
        Bottom-right: (1673, 559)


Top-left: (459, 2)
Bottom-right: (739, 469)

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from scipy.spatial.distance import cosine
import urllib.request
import os
from collections import deque


class PersonTracker:
    def __init__(self):
        self.model = YOLO('yolov8x.pt')

        # Download and load face cascade file
        cascade_file = 'haarcascade_frontalface_default.xml'
        if not os.path.exists(cascade_file):
            url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
            urllib.request.urlretrieve(url, cascade_file)

        self.face_cascade = cv2.CascadeClassifier(cascade_file)

        self.unique_persons = []
        self.position_history = []
        self.track_length = 30

        # Tracking parameters
        self.similarity_threshold = 0.7
        self.position_threshold = 150

        # Door zone parameters
        self.door_zone = None  # Will be set by user

    def process_video(self, video_path):
        # Initialize VideoCapture
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error: Unable to open video file: {video_path}")
            return

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video or failed to read frame.")
                break

            frame_count += 1
            print(f"Processing frame {frame_count}...")

            # Preprocess frame
            frame = cv2.resize(frame, (640, 384))

            # YOLO Inference
            try:
                results = self.model(frame)
            except Exception as e:
                print(f"Error during inference: {e}")
                break

            detections = results[0].boxes.data.cpu().numpy()
            print(f"Detections: {len(detections)}")

            # Draw detections
            for det in detections:
                x1, y1, x2, y2, conf, cls = det
                cv2.rectangle(
                    frame,
                    (int(x1), int(y1)),
                    (int(x2), int(y2)),
                    (0, 255, 0),
                    2
                )
                cv2.putText(
                    frame,
                    f"{conf:.2f}",
                    (int(x1), int(y1 - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    2
                )

            # Display frame
            cv2.imshow("Frame", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

tracker = PersonTracker()
unique_count = tracker.process_video(video_path)
print(f"Number of unique individuals detected: {unique_count}")